In [2]:
import pandas as pd
import numpy as np

In [37]:
message_data = pd.read_csv('../data/cleaned_messages.csv')

We extracted the week and year from each message's date and calculated the number of posts per week for each channel. We then computed the average number of posts per week per channel and merged this information back into the original message data. This allows us to analyze posting activity and compare channel engagement over time.

In [38]:
#average number of posts per week per channel 
message_data['week'] = pd.to_datetime(message_data['date']).dt.isocalendar().week
message_data['year'] = pd.to_datetime(message_data['date']).dt.isocalendar().year
weekly_posts = message_data.groupby(['year', 'week', 'channel_name']).size().reset_index(name='post_count')
# Calculate the average number of posts per week per channel
average_weekly_posts = weekly_posts.groupby('channel_name')['post_count'].mean().reset_index()
average_weekly_posts.rename(columns={'post_count': 'average_posts_per_week'}, inplace=True)
# Merge the average posts back into the original message data
message_data = pd.merge(message_data, average_weekly_posts, on='channel_name', how='left')
# Save the updated message data with post_ratio
print("top 5 channels with average posts per week...")
print(message_data.loc[:, ['channel_name', 'average_posts_per_week']].drop_duplicates().sort_values(by='average_posts_per_week', ascending=False).head())


top 5 channels with average posts per week...
        channel_name  average_posts_per_week
10994      @AwasMart               31.600000
5597       @qnashcom               25.402985
2625        @Leyueqa               22.240000
7811       @MerttEka               20.653846
0      @ZemenExpress               18.440000


**Average post view per channel**

In [41]:
message_data['average_views'] = message_data.groupby('channel_name')['views'].transform('mean')
print("channels with average views...")
print(message_data.loc[:, ['channel_name', 'average_views', 'average_posts_per_week']].drop_duplicates().sort_values(by='average_posts_per_week', ascending=False))

channels with average views...
                  channel_name  average_views  average_posts_per_week
10994                @AwasMart    3558.898734               31.600000
5597                 @qnashcom   19210.548766               25.402985
2625                  @Leyueqa   25583.372302               22.240000
7811                 @MerttEka   22509.325885               20.653846
0                @ZemenExpress    5201.516269               18.440000
5306     @modernshoppingcenter   17393.728522               15.315789
8348            @forfreemarket    9748.629412               14.166667
4330     @helloomarketethiopia    4112.091189               13.369863
7101              @gebeyaadama    4536.778873               12.033898
3181                 @sinayelj    7124.996047               11.000000
9415              @aradabrand2    4659.198020               10.821429
1626   @ethio_brand_collection   39796.206206               10.515789
9718               @marakisat2    2383.700155              

In [64]:
import re

def extract_price(text):
    if pd.isnull(text):
        return None
    # Look for price patterns, e.g., 1234, 1,234, 1234 birr, etc.
    match = re.search(r'(\d{1,3}(?:,\d{3})*|\d+)\s*(birr|br|ብር)', text, re.IGNORECASE)
    if match:
        return match.group(1).replace(',', '')
    return 0

#### Price of max viewed post products

In [65]:
message_group = message_data.loc[:, ['channel_name', 'average_views', 'average_posts_per_week', 'message', 'views']].groupby('channel_name').agg({
    'views': 'max',
    'message': lambda x: x.iloc[0] if len(x) > 0 else np.nan  

}).sort_values(by='views', ascending=False)
message_group['price'] = message_group['message'].apply(extract_price)
message_group


,views,message,price
channel_name,,,
@sinayelj,131282.0,Kids Hula Hoop In/Outdoor Fitness Collapsible ...,1200
@Leyueqa,118617.0,የፈሳሽ መጠጦች ማቅረቢያ 3 የተለያዩ መጠጦች በአንድ ላይ የየራሳቸዉ...,2800
@Shewabrand,114819.0,አልቆ የነበረዉ Under armour Curry 11 በድጋሚ አስገብተናል O...,0
@ethio_brand_collection,57101.0,"Skechers Ultra Lace Size 40,41,43 Price 3400...",3400
@Fashiontera,40998.0,"Made in Vietnam Size 41,42 Price 3400 ...",0
@qnashcom,40718.0,Knee Pain Relief Patch Specifically designed...,450
@MerttEka,35640.0,Microwave food cover ማይክሮ ዌቭ ውስጥ ምግብ ሲያሞቁ መ...,1150
@meneshayeofficial,32642.0,ከእስክሪን ነፃ የሆነ የቀለሜ መነሻ ጥቅል! ከእርሳስ አያያዝ ጀምሮ አማ...,0
@nevacomputer,28882.0,Dell Precision Workstation A powerful perfor...,0


In [66]:
# Calculate the average price per channel from the 'message_group' DataFrame
# Convert 'price' to numeric, coercing errors to NaN, then calculate the mean for each channel

# Ensure 'price' is numeric
message_group['price_numeric'] = pd.to_numeric(message_group['price'], errors='coerce')

# Calculate average price point (excluding zeros and NaNs)
average_price_point = message_group[message_group['price_numeric'] > 0]['price_numeric'].mean()

print(f"Average Price Point across channels: {average_price_point:.2f} birr")

# Optionally, show average price per channel
avg_price_per_channel = message_group.groupby(message_group.index)['price_numeric'].mean()
print("Average price per channel:")
print(avg_price_per_channel.dropna().sort_values(ascending=False))

Average Price Point across channels: 2281.82 birr
Average price per channel:
channel_name
@marakibrand               6000.0
@belaclassic               4400.0
@ethio_brand_collection    3400.0
@Leyueqa                   2800.0
@classybrands              2500.0
@aradabrand2               2500.0
@sinayelj                  1200.0
@MerttEka                  1150.0
@ZemenExpress               550.0
@qnashcom                   450.0
@modernshoppingcenter       150.0
@AwasMart                     0.0
@Fashiontera                  0.0
@Shewabrand                   0.0
@helloomarketethiopia         0.0
@forfreemarket                0.0
@gebeyaadama                  0.0
@meneshayeofficial            0.0
@marakisat2                   0.0
@kuruwear                     0.0
@nevacomputer                 0.0
Name: price_numeric, dtype: float64


In [73]:
# Calculate Lending Score: (Avg Views * 0.5) + (Posting Frequency * 0.5)
# Use average_views and average_posts_per_week from message_data, grouped by channel

lending_score_df = message_data.groupby('channel_name').agg({
    'average_views': 'first',
    'average_posts_per_week': 'first',
    'message': lambda x: x.iloc[0] if len(x) > 0 else np.nan    
}).reset_index()

lending_score_df['avg_price'] = lending_score_df['message'].apply(extract_price)
lending_score_df['lending_score'] = (
    lending_score_df['average_views'] * 0.5 +
    lending_score_df['average_posts_per_week'] * 0.5
)

# Sort by Lending Score descending
lending_score_df = lending_score_df.sort_values(by='lending_score', ascending=False)

lending_score_df[['channel_name', 'average_views', 'average_posts_per_week','avg_price', 'lending_score']]

,channel_name,average_views,average_posts_per_week,avg_price,lending_score
9,@ethio_brand_collection,39796.206206,10.515789,3400,19903.360998
2,@Leyueqa,25583.372302,22.240000,2800,12802.806151
3,@MerttEka,22509.325885,20.653846,1150,11264.989865
4,@Shewabrand,20403.752232,5.239766,0,10204.495999
19,@qnashcom,19210.548766,25.402985,450,9617.975876
17,@modernshoppingcenter,17393.728522,15.315789,150,8704.522156
7,@belaclassic,13462.801272,8.168831,4400,6735.485052
14,@marakibrand,11207.281369,5.844444,6000,5606.562907
1,@Fashiontera,10218.735714,2.485207,0,5110.610461
10,@forfreemarket,9748.629412,14.166667,0,4881.398039
